In [1]:
import threading
import cv2
from super_gradients.training import models
import time
from PIL import Image
from app import app, db, Count

[2023-06-01 14:50:38] INFO - utils.py - NumExpr defaulting to 8 threads.


In [2]:
def update_count_periodically_with_per(): #주기를 가지고 캡처
    with app.app_context():
        best_model = models.get('yolo_nas_s',
                        num_classes=3,
                        checkpoint_path="./Yolo_Nas/ckpt_best.pth")
        # 웹캠 열기
        video_capture = cv2.VideoCapture(0)

        # 캡쳐 주기 설정 (1초)
        capture_interval = 1

        # 이전 시간 저장 변수
        prev_time = time.time()
        loc_can_count = 0
        loc_pet_count = 0
        
        while True:
            # 주소 고정, 첫번째 행의 can, pet 개수 업데이트
            address = "서울특별시 구로구 가리봉동 121-30"
            lat= 37.482896
            lng=126.886887

            # 현재 시간
            curr_time = time.time()
            # 이미지 캡쳐
            ret, frame = video_capture.read()
            print(ret)


            # 일정 시간마다 캡쳐
            if curr_time - prev_time >= capture_interval:
                if ret and frame is not None:  # 정상적으로 프레임을 읽었을 경우에만 처리
                    # 이미지 파일로 저장
                    image_path = "./Yolo_Nas/captured_image.jpg"
                    cv2.imwrite(image_path, frame)

                    # 개수 세기
                    detection_pred = best_model.predict(image_path)._images_prediction_lst
                    pred = next(detection_pred).prediction
                    loc_can_count = 0
                    loc_pet_count = 0
                    for box, lab in zip(pred.bboxes_xyxy, pred.labels):
                        if lab == 1:
                            loc_can_count += 1
                            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 0, 255), 2)
                        elif lab == 2:
                            loc_pet_count += 1
                            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
                        elif lab == 0:
                            cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), -1)
                    
                    # 이미지 저장
                    output_path = "./static/image/output.png"
                    cv2.imwrite(output_path, frame)

                    # 이전 시간 업데이트
                    prev_time = curr_time
            # 웹캠 영상 출력
            cv2.imshow('Video', frame)

            count = Count.query.first()
            if count:
                count.can_count = loc_can_count
                count.pet_count = loc_pet_count
                db.session.commit()

            # 'q' 키를 누르면 종료
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        # 종료 시 리소스 해제
        video_capture.release()
        cv2.destroyAllWindows()

In [5]:
def update_count_periodically(cam_ind = 0): #되는대로 캡처
    #cam_ind : 웹캠 index (0 : 기본캠, 1 : 연결한 캠)
    with app.app_context():
        best_model = models.get('yolo_nas_s',
                        num_classes=3,
                        checkpoint_path="./Yolo_Nas/ckpt_best_fin.pth")
        # 웹캠 열기
        video_capture = cv2.VideoCapture(cam_ind)

        loc_can_count = 0
        loc_pet_count = 0
        
        while True:
            # 주소 고정, 첫번째 행의 can, pet 개수 업데이트
            address = "서울특별시 구로구 가리봉동 121-30"
            lat= 37.482896
            lng=126.886887

            # 현재 시간
            curr_time = time.time()
            # 이미지 캡쳐
            ret, frame = video_capture.read()
            print(ret)


            if ret and frame is not None:  # 정상적으로 프레임을 읽었을 경우에만 처리
                # 이미지 파일로 저장
                image_path = "./Yolo_Nas/captured_image.jpg"
                cv2.imwrite(image_path, frame)

                # 개수 세기
                detection_pred = best_model.predict(image_path)._images_prediction_lst
                pred = next(detection_pred).prediction
                loc_can_count = 0
                loc_pet_count = 0
                for box, lab in zip(pred.bboxes_xyxy, pred.labels):
                    if lab == 1:
                        loc_can_count += 1
                        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 0, 255), 2)
                    elif lab == 2:
                        loc_pet_count += 1
                        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
                    elif lab == 0:
                        cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (255, 0, 0), -1)

                # 이미지 저장
                output_path = "./static/image/output.png"
                cv2.imwrite(output_path, frame)

                # 이전 시간 업데이트
                prev_time = curr_time
            # 웹캠 영상 출력
            cv2.imshow('Video', frame)

            count = Count.query.first()
            if count:
                count.can_count = loc_can_count
                count.pet_count = loc_pet_count
                db.session.commit()

            # 'q' 키를 누르면 종료
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        # 종료 시 리소스 해제
        video_capture.release()
        cv2.destroyAllWindows()

In [ ]:
update_count_periodically(1)